In [1]:
import dspy

In [2]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/home/ec2-user/miniconda3/envs/dspy/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

In [4]:
dataset.train[0].items()

[('question',
  'At My Window was released by which American singer-songwriter?'),
 ('answer', 'John Townes Van Zandt')]

# Use Ollama
- Use the `Ollamalocal` in the dspy to interact with ollama model.

In [5]:
ollama_model = dspy.OllamaLocal(
    model='phi3',
    model_type='text',
    max_tokens=350,
    temperature=0.7,
    top_p=0.9,
    frequency_penalty=1.17,
    top_k=40
)

In [7]:
ollama_model("Tell me about the weather on pluto?")

[" Pluto, being a dwarf planet located in our solar system's Kuiper Belt, experiences extremely harsh and cold conditions. Its atmosphere is very thin compared to Earth's and mostly composed of nitrogen (N2), with traces of methane (CH4) and carbon monoxide (CO).\n\nThe weather on Pluto varies significantly due to its elliptical orbit around the Sun, which takes about 248 Earth years. During aphelion when it is farthest from the Sun, temperatures can drop as low as approximately -375 degrees Fahrenheit (-225 Celsius). This makes Pluto one of the coldest objects in our solar system.\n\nHowever, there are some interesting atmospheric phenomena on Pluto caused by its complex interactions with sunlight and moon Charon. Whenever Pluto moves closer to the Sun during its orbit (perihelion), it experiences a slight warming effect which can cause seasonal changes in temperature and possibly affect surface ice sublimation, contributing to atmospheric density variations as well.\n\nThe New Horizo

# Configure LLM 
- In order to use the ollama model, set the DsPy settings.

In [5]:
dspy.settings.configure(lm=ollama_model)

- The `Signature` is more like a `Task` that you want to be performed.
- The docstring is like `system` prompt.
- For the example below (QA) even the input and the output fields are defined.

In [6]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words.")

In [7]:
# Define predictor
generate_answer = dspy.Predict(BasicQA)

In [8]:
dev_example = devset[4]
pred = generate_answer(question=dev_example.question)
print(pred)

Prediction(
    answer="Earl Godwin's husband, King Edward the Confessor"
)


In [9]:
dev_example

Example({'question': 'In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?', 'answer': 'King Alfred the Great', 'gold_titles': {'Æthelweard (son of Alfred)', 'Ealhswith'}}) (input_keys={'question'})

In [13]:
ollama_model.inspect_history(1)




Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words.

---

Question: In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?
Answer: Earl Godwin's husband, King Edward the Confessor





"\n\n\nAnswer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: often between 1 and 5 words.\n\n---\n\nQuestion: In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?\nAnswer:\x1b Earl Godwin's husband, King Edward the Confessor\x1b\n\n\n"